In [13]:
# IMPORT
import pandas as pd
import numpy as np
import re

# Project-specific modules
from olist.data import *
from olist.utils import *
from olist.visualisation import *

# visualization
import matplotlib.pyplot as pl
import seaborn as sns
import folium

# Map number of orders by location

In [4]:
data = Olist().get_data()

In [5]:
sellers = data['sellers']
geolocation = data['geolocation']
customers = data['customers']

In [18]:
sellers_by_state = count_p_location(sellers, ['seller_state','seller_id'], 'state')
dummy(1,2)

NameError: name 'dummy' is not defined

In [ ]:
customers_by_state = location_data_process(customers, ['customer_state','customer_id'], 'Customers')

by_state_plot(customers_by_state)

In [ ]:
plot_map(sellers_by_zip_code)